In [1]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt 
from sklearn.neighbors import KNeighborsRegressor
from sklearn.feature_selection import RFE 
from sklearn.model_selection import train_test_split, cross_val_score 
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold
from sklearn.model_selection import RepeatedKFold

wdir =  'C:/jupyter_devel/kist-europe/QSAR/AOP_data/'
csv = '5-alpha-reductase-maccs-remcols-stdval-dltnan.csv'
#csv = '5-alpha-reductase-maccs-remcols-stdval-dlt-outlier.csv'

#save dir for model result
result_wdir = 'C:/jupyter_devel/kist-europe/QSAR/AOP_data/model_result/'

df = pd.read_csv(wdir+csv)

# check which compound is NaN Standard Value 
"""
null_idx = []
df_std = df['Standard Value']
df_null = df_std.isnull()
for i in range(len(df_null)):
    if df_null.iloc[i] == True: 
        null_idx.append(i)
null_idx
"""

"\nnull_idx = []\ndf_std = df['Standard Value']\ndf_null = df_std.isnull()\nfor i in range(len(df_null)):\n    if df_null.iloc[i] == True: \n        null_idx.append(i)\nnull_idx\n"

In [2]:
df = df.dropna(axis=0) #axis=0 means forward rows 

y = df['Standard Value']
x = df.drop(['Molecule','Standard Value'],axis=1) #axis=1 means forward cols

#save data with removing NaN values
#df.to_csv(wdir+'5-alpha-reductase-maccs-remcols-stdval-dltnan.csv'.format(i),index=True)

-----

In [ ]:
def bootstrapping (datasize,x, y):
    #Random sampling with duplication
    train = []
    for i in range(datasize):
        train.append(random.randrange(datasize))
    
    #Find indices not sampled
    test = []
    for i in range(df.shape[0]):
        if i not in train:
            test.append(i)
    
    #Subsetting dataframe from sampled and unsampled data
    train_x = x.iloc[train]
    train_y = y.iloc[train]
    
    test_x = x.iloc[test]
    test_y = y.iloc[test]
    
    return train_x, train_y, test_x, test_y

-----

## version 0 _k folds      for getting knowledge that which data is involving within certain fold

### for modern stdval prediction

In [ ]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = test_size, shuffle = True)

In [ ]:

#fold setting
kf = RepeatedKFold(n_splits = 5,n_repeats=10)
kf.get_n_splits(x_train)
#kf
#>>> KFold(n_splits=5, random_state=None, shuffle=False)
#>>> <sklearn.model_selection._split.RepeatedKFold at 0x2939c6ffcc8>

#for numbering k folds
i=1
cv_mean = []

#DataFrame generate
y_pred_result = pd.DataFrame()

#model generate
neigh = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')

#cross-val training
for train_idx, test_idx in kf.split(x_train):
    print("train : ", train_idx, "test : ", test_idx)
    x_cv_train , x_cv_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
    y_cv_train , y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    neigh.fit(x_cv_train,y_cv_train)
    
    y_cv_testidx = test_idx
    y_cv_pred = neigh.predict(x_cv_test)
    cv_mse.append(mean_squared_error(y_cv_test,y_cv_pred))
    
    y_pred_result['y_cv_testidx_{}'.format(i)] = y_cv_testidx
    y_pred_result['y_cv_pred_{}'.format(i)] = y_cv_pred
    i += 1 
    if i % 5 == 0 : 
        print(cv_mean)
        

#model retrain with all train data  
neigh.fit(x_train,y_train) # > model train
y_test_pred = neigh.predict(x_test) # > model predidction

# 
y_test_pred_df = pd.DataFrame(y_test_pred,columns=['y_pred_test'])
y_pred_result = pd.concat([y_pred_result, y_test_pred_df], axis=1, ignore_index=False)

#y_pred_result.to_csv(result_wdir+'y_pred_result.csv'.format(i),index=True)
y_pred_result

In [ ]:
MSE_v0 = mean_squared_error(y_test,y_test_pred)
MSE_v0
RMSE_v0 = np.sqrt(MSE_v0)
RMSE_v0

print("MSE : {:.2f} | RMSE : {:.2f} ".format(MSE_v0,RMSE_v0))

### for log scale stdval prediction

In [ ]:
y_log = np.log(y)
y_log

#for saving values
#y_log.to_csv(result_wdir+'LOG(Standard_Value).csv',index=False)

In [ ]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y_log,test_size = test_size, shuffle = False)

In [ ]:

#fold setting
kf = KFold(n_splits = 5)
kf.get_n_splits(x_train)
#kf
#>>> KFold(n_splits=5, random_state=None, shuffle=False)

#for numbering k folds
i=1

#DataFrame generate
y_log_pred_result = pd.DataFrame()

#model generate
neigh = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')

#cross-val training
for train_idx, test_idx in kf.split(x_train):
    print("train : ", train_idx, "test : ", test_idx)
    x_cv_train , x_cv_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
    y_cv_train , y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    neigh.fit(x_cv_train,y_cv_train)
    
    y_cv_testidx = test_idx
    y_cv_pred = neigh.predict(x_cv_test)
    y_log_pred_result['y_cv_testidx_{}'.format(i)] = y_cv_testidx
    y_log_pred_result['y_cv_pred_{}'.format(i)] = y_cv_pred
    i += 1 

#model retrain with all train data  
neigh.fit(x_train,y_train) # > model train
y_log_test_pred = neigh.predict(x_test) # > model predidction

# 
y_log_test_pred_df = pd.DataFrame(y_log_test_pred,columns=['y_log_test_pred'])
y_log_pred_result = pd.concat([y_log_pred_result, y_log_test_pred_df], axis=1, ignore_index=False)

y_log_pred_result.to_csv(result_wdir+'y_log_pred_result.csv',index=True)


In [ ]:
MSE_v0_ = mean_squared_error(y_test,y_log_test_pred)
MSE_v0_
RMSE_v0_ = np.sqrt(MSE_v0_)
RMSE_v0_

print("MSE : {:.2f} | RMSE : {:.2f} ".format(MSE_v0_,RMSE_v0_))

## for pIC50 value

In [ ]:
y

In [3]:
#pic50 = log(10^9/ic50) = 9 - log10(ic50)

y_pic50 = 9 - np.log10(y)
y_pic50

#for saving values
y_pic50.to_csv(result_wdir+'5-alpha-reductase_dltnan_PIC50_value.csv',index=False)

C:\Users\chon0\Anaconda3\envs\develroom\lib\site-packages\ipykernel_launcher.py:7: FutureWarning: The signature of `Series.to_csv` was aligned to that of `DataFrame.to_csv`, and argument 'header' will change its default value from False to True: please pass an explicit value to suppress this warning.
  import sys


In [4]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y_pic50,test_size = test_size, shuffle = True)

In [5]:
x_test_idx = []
for row in x_test.index:
    x_test_idx.append(row)
    
x_test_idx

[9,
 86,
 177,
 60,
 138,
 186,
 190,
 188,
 152,
 184,
 113,
 172,
 193,
 169,
 203,
 134,
 121,
 258,
 126,
 260,
 146,
 73,
 128,
 77,
 269,
 49,
 183,
 25]

### model train 

In [6]:

#fold setting
kf = RepeatedKFold(n_splits = 5,n_repeats=10)
kf.get_n_splits(x_train)
#kf
#>>> KFold(n_splits=5, random_state=None, shuffle=False)

#for numbering k folds
i=0
cv_rmse = []
cv_rmse_mean = []
tmp = []

#DataFrame generate
y_pic_pred_result = pd.DataFrame()
y_pic_pred_result['idx'] = pd.Series([i for i in range(len(y))])


#model generate
neigh = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')

#cross-val training
for train_idx, test_idx in kf.split(x_train):
    print("iteration num : {}".format(i))
    print("train : ", train_idx, "\ntest : ", test_idx)
    x_cv_train , x_cv_test = x_train.iloc[train_idx], x_train.iloc[test_idx]
    y_cv_train , y_cv_test = y_train.iloc[train_idx], y_train.iloc[test_idx]
    
    neigh.fit(x_cv_train,y_cv_train)
    
    y_cv_testidx = test_idx
    y_cv_pred = neigh.predict(x_cv_test)
    cv_rmse.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    cv_rmse_mean.append(round(np.sqrt(mean_squared_error(y_cv_test,y_cv_pred)),4))
    
    y_pic_pred_result['y_cv_testidx_{}'.format(i)] = pd.Series()
    y_pic_pred_result['y_cv_pred_{}'.format(i)] = pd.Series()
    
    for j in range(len(y_cv_testidx)):
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_testidx_{}'.format(i)] = y_cv_testidx[j]
        y_pic_pred_result.loc[y_cv_testidx[j],'y_cv_pred_{}'.format(i)] = y_cv_pred[j]
        
    
    
    #y_pic_pred_result['y_cv_testidx_{}'.format(i)] = pd.Series(y_cv_testidx)
    #y_pic_pred_result['y_cv_pred_{}'.format(i)] = pd.Series(y_cv_pred)
    i += 1 
    
    if i % 5 == 0 : 
        tmp.append(np.mean(cv_rmse_mean))
        print(cv_rmse_mean) # per fold rmse value in one cv 
        print("mean : ",np.mean(cv_rmse_mean)) # RMSE mean value per iteration of cv  
        cv_rmse_mean.clear()
        

print("total fold mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total cv mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 

#model retrain with all train data  
neigh.fit(x_train,y_train) # > model train
y_pic_test_pred = neigh.predict(x_test) # > model predidction

# 
y_pic_test_pred_df = pd.DataFrame(columns=['y_pic_test_idx','y_pic_test_pred'])
for j in range(len(x_test_idx)):
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_idx'] = x_test_idx[j]
        y_pic_test_pred_df.loc[x_test_idx[j],'y_pic_test_pred'] = y_pic_test_pred[j]
#y_pic_test_pred_df = pd.DataFrame(y_pic_test_pred,columns=['y_pic_test_pred'])
y_pic_pred_result = pd.concat([y_pic_pred_result, y_pic_test_pred_df], axis=1, ignore_index=False)

#save csv 
y_pic_pred_result.to_csv(result_wdir+'5-alpha-reductase_dltnan_y_pic_pred_result.csv',index=True)


iteration num : 0
train :  [  1   2   5   6   7   8   9  10  13  16  17  18  19  20  21  22  23  24
  25  26  27  28  29  31  33  34  36  37  38  40  41  42  43  44  45  47
  48  49  51  53  54  55  56  57  58  59  60  61  63  64  66  67  69  70
  71  72  75  76  77  78  80  81  83  84  85  86  87  88  89  91  92  93
  94  95  96  97  98  99 100 101 102 103 104 105 106 110 111 112 113 115
 117 118 120 122 123 124 125 128 129 130 132 133 135 136 137 138 139 140
 141 142 143 144 145 146 147 148 149 151 152 153 154 155 156 157 158 159
 160 161 162 163 164 165 166 167 168 169 170 171 172 173 174 175 176 177
 178 179 181 182 183 184 185 187 188 189 191 192 193 195 196 197 198 199
 201 202 203 205 206 207 208 209 212 213 214 216 218 219 220 221 222 223
 224 225 226 228 229 231 232 233 234 235 236 237 238 241 243 244 245 247
 248 249] 
test :  [  0   3   4  11  12  14  15  30  32  35  39  46  50  52  62  65  68  73
  74  79  82  90 107 108 109 114 116 119 121 126 127 131 134 150 180 186
 190 

iteration num : 9
train :  [  0   3   5   6   7   8   9  10  11  12  14  15  17  19  20  21  22  23
  24  25  26  27  28  29  30  31  32  33  34  35  36  37  38  40  42  44
  45  46  47  48  49  50  51  52  54  56  57  58  59  61  62  63  64  66
  67  69  70  71  72  73  74  75  76  78  80  81  82  83  84  85  86  89
  90  91  92  93  94  95  96  97  98 100 101 102 104 105 106 107 108 109
 110 111 112 113 114 115 117 118 119 120 121 122 123 125 126 127 128 130
 132 133 134 135 136 137 138 139 140 141 142 143 144 145 146 147 148 149
 150 154 155 156 157 158 159 160 161 162 163 164 167 169 170 171 172 173
 174 175 176 177 181 183 184 186 187 189 190 191 192 194 195 197 198 199
 201 203 204 205 206 207 209 211 212 213 214 216 217 218 219 220 221 222
 223 224 225 226 228 230 231 232 233 234 235 236 238 239 242 243 245 247
 248 249] 
test :  [  1   2   4  13  16  18  39  41  43  53  55  60  65  68  77  79  87  88
  99 103 116 124 129 131 151 152 153 165 166 168 178 179 180 182 185 188
 193 

iteration num : 17
train :  [  1   2   3   6   7   8   9  10  11  12  13  14  15  16  17  20  21  22
  23  24  25  26  27  28  29  30  31  32  33  35  36  37  38  39  40  41
  43  44  46  47  48  49  50  51  52  53  56  58  59  60  61  62  63  64
  65  66  67  68  69  70  71  72  73  74  75  76  77  78  80  82  83  84
  86  87  88  89  90  92  93  94  96  97  99 100 103 105 106 108 109 111
 113 114 115 116 117 118 119 120 122 123 124 125 127 128 129 130 132 133
 134 135 136 141 142 144 146 147 148 149 150 151 152 153 154 155 156 157
 159 160 161 162 163 164 165 166 167 168 169 170 173 174 175 176 177 178
 179 180 181 183 184 185 186 187 188 190 191 192 193 194 195 196 197 198
 199 200 201 202 203 207 208 209 210 212 214 216 217 218 220 221 222 223
 224 225 227 228 229 230 231 233 234 235 238 239 240 241 242 244 245 246
 247 248] 
test :  [  0   4   5  18  19  34  42  45  54  55  57  79  81  85  91  95  98 101
 102 104 107 110 112 121 126 131 137 138 139 140 143 145 158 171 172 182
 189

iteration num : 26
train :  [  0   3   4   5   6   7  10  11  12  13  14  15  16  18  19  21  22  23
  24  25  26  28  29  30  31  32  33  34  35  36  39  40  41  42  44  46
  48  49  50  51  52  53  54  56  59  61  62  63  65  66  67  69  70  71
  72  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89
  90  91  93  94  95  96  97  98  99 100 101 102 104 105 108 110 111 112
 113 114 115 116 119 121 123 124 125 126 127 128 130 131 132 133 134 135
 136 139 140 141 145 146 147 148 149 150 152 154 155 156 157 159 160 161
 162 163 164 165 166 167 168 169 170 171 172 173 176 177 178 180 181 183
 184 185 186 187 189 190 191 193 194 195 196 197 198 199 200 201 202 203
 204 205 207 208 209 210 211 212 213 214 215 216 217 218 219 221 222 223
 224 225 226 227 228 229 231 232 233 235 236 237 238 239 240 244 245 246
 247 249] 
test :  [  1   2   8   9  17  20  27  37  38  43  45  47  55  57  58  60  64  68
  92 103 106 107 109 117 118 120 122 129 137 138 142 143 144 151 153 158
 174

[0.8044, 0.703, 0.7914, 0.894, 0.6507]
mean :  0.7687
iteration num : 35
train :  [  0   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18  19
  20  21  22  23  25  26  27  29  30  31  32  33  34  36  38  39  40  41
  42  44  45  46  47  48  49  52  53  55  56  57  58  60  61  62  63  64
  65  66  68  69  70  71  75  76  77  78  79  80  81  82  83  84  85  86
  87  89  90  91  92  93  94  96  97  99 100 103 104 106 108 109 110 111
 112 113 114 115 116 117 118 119 120 121 123 124 125 126 127 128 129 130
 131 132 134 135 137 138 139 140 141 143 144 145 148 149 152 154 155 156
 157 160 161 163 164 165 167 168 170 171 172 174 176 178 179 181 182 183
 184 185 186 187 188 189 191 192 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 211 212 213 214 215 216 217 218 220 222 224 225
 226 227 228 229 230 231 232 233 234 237 239 240 241 242 243 244 245 247
 248 249] 
test :  [  1   2  24  28  35  37  43  50  51  54  59  67  72  73  74  88  95  98
 101 102 105 107 122 13

iteration num : 44
train :  [  0   1   2   4   5   7   8   9  10  11  12  14  15  16  17  18  19  20
  21  22  23  25  26  27  28  29  30  31  32  33  34  35  36  37  38  39
  40  41  42  44  49  50  51  52  53  55  56  58  59  60  61  62  64  65
  66  67  68  69  71  72  73  74  75  77  78  79  80  81  82  83  85  87
  88  90  91  92  93  94  96  97  98  99 100 101 102 103 104 105 106 107
 108 109 110 111 112 114 115 118 120 121 122 124 127 128 130 131 133 134
 135 136 138 140 141 142 145 146 147 148 149 150 151 152 153 154 156 157
 159 160 161 162 164 165 166 167 168 169 170 171 172 173 174 175 177 178
 179 180 182 183 184 187 188 189 190 192 193 194 195 197 200 202 203 204
 205 206 207 208 210 211 212 213 214 215 217 218 220 221 224 225 226 227
 228 229 230 231 232 233 234 236 237 238 239 240 242 243 244 245 246 247
 248 249] 
test :  [  3   6  13  24  43  45  46  47  48  54  57  63  70  76  84  86  89  95
 113 116 117 119 123 125 126 129 132 137 139 143 144 155 158 163 176 181
 185

### getting validation result

In [20]:
#RESULT
print("total fold rmse mean : ", round(np.mean(cv_rmse),4)) # mean of 50 cv rmse (1회 iter돌 때의 모든 rmse의 평균 (50회의 평균)) 
print("total cv rmse mean : ",round(np.mean(tmp),4)) # mean of cv's mean of 1 iter (각 cv rmse 평균의 평균) # we need var of this value 
print("each cv rmse average : \n{} \nvariance of rmse of every folds : {} ".format(tmp,round(np.var(cv_rmse),4)))
print("train data size : {} ({}%)\n".format(len(x_train),round((len(x_train)/len(x)),2)), "test data size : {} ({}%)".format(len(x_test),round((len(x_test)/len(x)),2)))

total fold rmse mean :  0.7626
total cv rmse mean :  0.7626
each cv rmse average : 
[0.75148, 0.7835799999999999, 0.74776, 0.7918, 0.7440399999999999, 0.7497399999999999, 0.7687, 0.7808400000000001, 0.75098, 0.7567] 
variance of rmse of every folds : 0.0121 
train data size : 250 (0.9%)
 test data size : 28 (0.1%)


### get inverse pic , predicted Standard Value

In [ ]:
y_pic = y_pic_pred_result['y_pic_test_pred']
y_compare = np.power(10, (9-y_pic))
print(y[250:], y_compare[:28])

### get mse and rmse of test set

In [21]:
MSE_v0_3 = mean_squared_error(y_test,y_pic_test_pred)
MSE_v0_3
RMSE_v0_3 = np.sqrt(MSE_v0_3)
RMSE_v0_3

print("MSE : {:.2f} | RMSE : {:.2f} ".format(MSE_v0_3,RMSE_v0_3))

MSE : 0.58 | RMSE : 0.76 


# version 1 _spliting data for train as 0.9 and test as 0.1 / get rmse 

In [ ]:
test_size = 0.1 
x_train, x_test, y_train, y_test = train_test_split(x,y,test_size = test_size, shuffle = True)

In [ ]:
neigh = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')
neigh.fit(x_train,y_train)

In [ ]:
neigh_pred_test = pd.DataFrame(neigh.predict(x_test),columns=['neigh_pred_test'])
neigh_pred_test

##  1 - 1) RMSE with data splitting 

In [ ]:
MSE = mean_squared_error(y_test,neigh_pred_test)
MSE
RMSE = np.sqrt(MSE)
RMSE
log_RMSE = np.log(RMSE)
log_RMSE

print("MSE : {:.2f} | RMSE : {:.2f} | log RMSE : {:.2f}".format(MSE, RMSE, log_RMSE))

##  1 - 2) RMSE with cv  

In [ ]:
#cross validation
cv=5
neg_mse_cv = cross_val_score(neigh,
                            x,
                            y,
                            scoring='neg_mean_squared_error',
                            cv=cv)
mse_cv_ = (-1)*neg_mse_cv
rmse_cv_ = np.sqrt(mse_cv_)
log_rmse_cv_ = np.log(rmse_cv_)

log_rmse_cv_mean = np.sum(log_rmse_cv_)/cv
log_rmse_cv_mean

##  #1 - demo 

In [ ]:
num = 94
test_x = []
test_x = x.loc[num][:]

In [ ]:
#target value
# >>> y.loc[num]
# predict value 
# >>> neigh.predict([test_x])
print('target value : {}\npredict value : {}'.format(y.loc[num],neigh.predict([test_x])))

In [ ]:
neigh.score(x,y)

# version 2 _using log scaled data

In [ ]:
y

In [ ]:
y_log = []
for i in range(len(y)):
    y_log.append(np.log(y.loc[i]))
    
y_log_df = pd.DataFrame(y_log,columns=['stdval_log'])
y_log_df

In [ ]:
test_size = 0.1 
x_train_v2, x_test_v2, y_train_v2, y_test_v2 = train_test_split(x,y_log_df,test_size = test_size, shuffle = True)

In [ ]:
neigh_v2 = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')
neigh_v2.fit(x_train_v2,y_train_v2)

In [ ]:
y_test_v2

In [ ]:
neigh_v2_pred_test = pd.DataFrame(neigh_v2.predict(x_test_v2),columns=['neigh_pred_test'])
neigh_v2_pred_test

##  2 - 1) RMSE with data splitting  

In [ ]:
MSE_v2 = mean_squared_error(y_test_v2,neigh_v2_pred_test)
MSE_v2
RMSE_v2 = np.sqrt(MSE_v2)
RMSE_v2
log_RMSE_v2 = np.log(RMSE_v2)
log_RMSE_v2

print("MSE : {:.2f} | RMSE : {:.2f} | log RMSE : {:.2f}".format(MSE_v2, RMSE_v2, log_RMSE_v2))
print("in this case, log RMSE is no meaning bacause we already used log scaled data")

##  2 - 2) RMSE with cv  

In [ ]:
#cross validation
cv=5
neg_mse_cv_v2 = cross_val_score(neigh_v2,
                            x,
                            y_log_df,
                            scoring='neg_mean_squared_error',
                            cv=cv)
mse_cv_v2 = (-1)*neg_mse_cv_v2
rmse_cv_v2 = np.sqrt(mse_cv_v2)
#log_rmse_cv_v2 = np.log(rmse_cv_v2)

#log_rmse_cv_mean_v2 = np.sum(log_rmse_cv_v2)/cv
#og_rmse_cv_mean_v2

rmse_cv_v2_mean = np.sum(rmse_cv_v2)/cv
rmse_cv_v2_mean

# version 3 _std ->log(std) -> model -> log(pred) -> pred 

In [ ]:
y_log = []
for i in range(len(y)):
    y_log.append(np.log(y.loc[i]))
    
y_log_df = pd.DataFrame(y_log,columns=['stdval_log'])
y_log_df

In [ ]:
test_size = 0.1 
x_train_v3, x_test_v3, y_train_v3, y_test_v3 = train_test_split(x,y_log_df,test_size = test_size, shuffle = True)

In [ ]:
neigh_v3 = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')
neigh_v3.fit(x_train_v3,y_train_v3)

In [ ]:
neigh_v3_pred_test = pd.DataFrame(neigh_v3.predict(x_test_v3),columns=['nei_pred_test_v3'])
neigh_v3_pred_test_exp = np.exp(neigh_v3_pred_test)
neigh_v3_pred_test_exp

In [ ]:
y_test_v3_exp = np.exp(y_test_v3)
y_test_v3_exp

## 3 - 1) RMSE with data splitting

In [ ]:
MSE_v3 = mean_squared_error(y_test_v3_exp,neigh_v3_pred_test_exp)
MSE_v3
RMSE_v3 = np.sqrt(MSE_v3)
RMSE_v3
log_RMSE_v3 = np.log(RMSE_v3)
log_RMSE_v3

print("MSE : {:.2f} | RMSE : {:.2f} | log RMSE : {:.2f}".format(MSE_v3, RMSE_v3, log_RMSE_v3))
#print("in this case, log RMSE is no meaning bacause we already used log scaled data")

## 하단코드 적용 불가

In [ ]:
#cross validation
cv=5
neg_mse_cv_v2 = cross_val_score(neigh_logdata_v2,
                            x,
                            y_log_df,
                            scoring='neg_mean_squared_error',
                            cv=cv)
mse_cv_v2 = (-1)*neg_mse_cv_v2
rmse_cv_v2 = np.sqrt(mse_cv_v2)
#log_rmse_cv_v2 = np.log(rmse_cv_v2)

#log_rmse_cv_mean_v2 = np.sum(log_rmse_cv_v2)/cv
#og_rmse_cv_mean_v2

rmse_cv_v2_mean = np.sum(rmse_cv_v2)/cv
rmse_cv_v2_mean

# version4 _feature selection

In [ ]:
from sklearn.feature_selection import SelectKBest, f_regression

In [ ]:
x.shape
x_new = SelectKBest(f_regression,k=30).fit_transform(x,y)

In [ ]:
x_new.shape

In [ ]:
test_size = 0.1 
x_train_v4, x_test_v4, y_train_v4, y_test_v4 = train_test_split(x_new,y,test_size = test_size, shuffle = True)

neigh_v4 = KNeighborsRegressor(n_neighbors = 5,metric='jaccard',weights = 'distance')
neigh_v4.fit(x_train_v4,y_train_v4)

In [ ]:
nei_pred_v4 = pd.DataFrame(neigh_v4.predict(x_test_v4),columns=['pred_v4'])

In [ ]:
mse_v4 = mean_squared_error(y_test_v4,nei_pred_v4)
log_mse_v4 = np.log(mse_v4)
log_mse_v4

##### 값이 더 크다

------

In [ ]:
#cross validation
cv=5
neg_mse_cv_v4 = cross_val_score(neigh_v4,
                            x_new,
                            y,
                            scoring='neg_mean_squared_error',
                            cv=cv)
mse_cv_v4 = (-1)*neg_mse_cv_v4
rmse_cv_v4 = np.sqrt(mse_cv_v4)
log_rmse_cv_v4 = np.log(rmse_cv_v4)

log_rmse_cv_mean_v4 = np.sum(log_rmse_cv_v4)/cv
log_rmse_cv_mean_v4

#rmse_cv_v4_mean = np.sum(rmse_cv_v4)/cv
#rmse_cv_v4_mean

##### 얘는 비슷. 더 나은 feature selection 필요 

In [ ]:
dict = {'a':[1,2,np.nan,4], 'b':[5,np.nan,7,8]}
dict_df = pd.DataFrame(dict)

dict_df

In [ ]:
dict_df.dropna(subset=['a'])